# Settings

### Import libraries

In [1]:
import os
from typing import Union
from langfuse import Langfuse
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_ollama import ChatOllama
from langfuse.callback import CallbackHandler
from langchain_core.messages import HumanMessage

### Initialize the LLM

In [2]:
ollama = ChatOllama(
    base_url="http://ollama:11434",
    model="llama3.2",
    temperature=0.7
)

### Connecting with LangFuse

In [3]:
langfuse = Langfuse()
langfuse_handler = CallbackHandler()

### Defining tools

In [4]:
@tool
def math_operation(
        a: Union[int, float],
        b: Union[int, float],
        operation: str,
):
    """
    Perform a mathematical operation on two numbers.

    Args:
        a (Union[int, float]): The first number in the operation.
        b (Union[int, float]): The second number in the operation.
        operation (str): A string representing the mathematical operation.
                          Supported operations: '+', '-', '*', '/'.

    Returns:
        Union[int, float]: The result of the operation, which will be an integer or float depending on the input.

    Raises:
        ValueError: If the operation is not a valid mathematical operator.
        ZeroDivisionError: If division by zero is attempted.
    
    Example:
        >>> math_operation(a=3, b=4, operation="-")
        -1
        >>> math_operation(a=5, b=2, operation="/")
        2.5
    """
    op: str = f"{a}{operation}{b}"
    return eval(op)

### Adding tools to the LLM

In [5]:
ollama_enhanced = ollama.bind_tools(my_tools:=[math_operation])
tool_node = ToolNode(my_tools)

## Using the model

In [6]:
tool_call = ollama_enhanced.invoke(
    [HumanMessage(content=f"Me acabo de acordar que tenia 5 platano, pero ahora solo veo 2 😔 ¿cuantos me hacen falta?")], 
    config={"callbacks": [langfuse_handler]}
)

In [7]:
response = tool_node.invoke({"messages": [tool_call]})

In [8]:
response

{'messages': [ToolMessage(content='7', name='math_operation', tool_call_id='1c77754a-1488-4f57-970c-e32d872df119')]}